In [1]:
from secrets import *

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import numpy as np
import pandas as pd
import seaborn as sns
import json
import time
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')

In [4]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, 
    client_id=SPOTIPY_CLIENT_ID, 
    client_secret=SPOTIPY_CLIENT_SECRET,
    username=CLIENT_USERNAME,
    redirect_uri=SPOTIPY_REDIRECT_URI))

In [66]:
def extract_playlists_from_user():
    """
    Returns a dictionary of playlist URIs and their corresponding titles.
    """
    playlists = sp.current_user_playlists()
    d = {}
    for playlist in playlists['items']:
        d[playlist["uri"]] = playlist['name']
    
    return d

In [67]:
def extract_songs_from_playlist(uri):
    """
    Returns a dictionary of track URIs and their corresponding names given a playlist.
    """
    playlist = sp.playlist_tracks(uri)
    playlist_d = {}
    for song in playlist['items']:
        playlist_d[song['track']['id']] = song['track']['name']
    
    return playlist_d

In [68]:
def create_playlist_df(songs: dict):
    """
    Returns a DataFrame representation of a given playlist.
    """
    column_labels = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 
                 'analysis_url', 'duration_ms', 'time_signature']

    features_df = pd.DataFrame(columns=column_labels)
    for (uri, title) in zip(songs.keys(), songs.values()):
        features = sp.audio_features([uri])[0]
        features_df = features_df.append(pd.DataFrame(data=features, index=[title]), sort=False)
    
    return features_df

In [73]:
def create_df_from_uri(uri):
    """
    Ensemble function that combines songs extraction and DataFrame creation.
    """
    songs = extract_songs_from_playlist(uri)
    df = create_playlist_df(songs)
    
    return df

In [95]:
playlists = extract_playlists_from_user()

In [94]:
df_dict = {}
for (uri, name) in zip(playlists.keys(), playlists.values()):
    new_df = create_df_from_uri(uri)
    new_df.to_csv(name, index=True)
    df_dict[name] = new_df